In [1]:
# A notebook for organising AI progress metrics

from datetime import date

# We have the following structures:
#
# problem 
#     \   \
#      \   metrics  -  measures 
#       \
#        - subproblems
#             \
#           metrics
#              \
#             measures
#
# problems are tagged with attributes:
# eg, vision, abstract-games, language, world-modelling, safety
#     agi       -- most capable humans can do this, the problem is AGI-complete
#     super     -- the very best humans can do this, or human organisations have solved this
#     verysuper -- neither humans nor human orgs have solved this
#
# problems can have "subproblems", including simpler cases and preconditions

class Problem:
    def __init__(self, name, attributes=[], solved=False):
        self.name = name
        self.attributes = attributes
        self.subproblems = []
        self.superproblems = []
        self.metrics = []
        
    def subproblem(self, other_problem):
        self.superproblems.append(other_problem)
        other_problem.subproblems.append(self)
        
    def metric(self, *args, **kwargs):
        m = Metric(*args, **kwargs)
        self.metrics.append(m)
        return m

class Metric:
    def __init__(self, name, url=None, solved=False):
        self.name = name
        self.measures = []
        self.solved = solved
        self.url = url
        
    def measure(self, *args, **kwargs):
        m = Measurement(*args, **kwargs)
        self.measures.append(m)
        # Add logic for detection solutions at this point
        return m
        
class Measurement:
    def __init__(self, date, value, name, url):
        self.date = date
        self.value = value
        self.name = name
        self.url = url


In [ ]:
# BEGIN ACTUALLY CLASSIFYING PROBLEMS

scene_description = Problem("Scene description", ["agi", "vision", "language", "world-modelling"])
image_classification = Problem("Image classification", ["vision", "agi"])
scene_description.subproblem(image_classification)

imagenet = image_classification.metric("imagenet", "http://image-net.org")
imagenet.measure(date(2010,8,31), 0.28191, "NEC UIUC", "http://image-net.org/challenges/LSVRC/2010/results")
"""
** 2010: 0.28191**
**NEC UIUC**
http://image-net.org/challenges/LSVRC/2010/results

** 2011: 0.25770
 XRCE**

** 2012: 0.16422**
** Supervision**
http://image-net.org/challenges/LSVRC/2012/results.html

** 2013: 0.11743 **
**Clarifai**
http://www.image-net.org/challenges/LSVRC/2013/results.php

** 2014: 0.07405**
**VGG**
http://image-net.org/challenges/LSVRC/2014/index
 

**2015: 0.03567**
**MSRA**
http://image-net.org/challenges/LSVRC/2015/results

** 2016: 0.02991**
**Trimps-Soushen**
http://image-net.org/challenges/LSVRC/2016/results
* * *
"""
None